# EPM Analysis
- Behavior is 30 fps, cell activity is 20 fps for drd73, drd77 and 10 fps for drd87, drd46

In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.style as style
import seaborn as sns
import plotly
import plotly.figure_factory as ff
import plotly.graph_objs as go
import Core
import analysis_utils

from scipy import stats
from IPython.core.interactiveshell import InteractiveShell

plotly.offline.init_notebook_mode(connected=True);
InteractiveShell.ast_node_interactivity = "all"

%matplotlib inline

In [2]:
data = pd.read_csv('~/TraceRaw_Mouse_drd46.csv', header=None)

# returns [cell_data, cell_AUC_df, cell_transients]
lst_of_dataframes = Core.detect_ca_transients_mossy(data, 2, 0.5, 0.2, 10)

lst_of_dataframes[0].columns = ['neuron' + str(i) for i in range(1, len(lst_of_dataframes[2].columns)+1)]
lst_of_dataframes[1].columns = ['neuron' + str(i) for i in range(1, len(lst_of_dataframes[2].columns)+1)]
lst_of_dataframes[2].columns = ['neuron' + str(i) for i in range(1, len(lst_of_dataframes[2].columns)+1)]

z_scored_dataframe = lst_of_dataframes[0]
AUC_dataframe = lst_of_dataframes[1]
cell_transients_dataframe = lst_of_dataframes[2]

# cell_transients_dataframe.plot(subplots=True, figsize=(15, 15), sharex=True, sharey=True, rot=0);

In [3]:
behavior_column_names = ['Trial_time', 'Recording_time', 'X_center', 'Y_center', 'Area', 'Areachange', 
                         'Elongation', 'Distance_moved', 'Velocity', 'Arena_centerpoint',
                         'Open1_centerpoint', 'Open2_centerpoint',
                         'Closed1_centerpoint', 'Closed2_centerpoint',
                         'OpenArms_centerpoint', 'ClosedArms_centerpoint', 'Result_1']

activity_df = pd.read_csv('~/Hen_Lab/Mice/drd46_experiments/activity_drd46.csv', header=None)
behavior_df = pd.read_csv('~/Hen_Lab/Mice/drd46_experiments/behavior_drd46.csv', header=None)

# Save only every nth row in order to downsample behavior Dataframes from 30fps -> 10fps
ROW_MULTIPLE = 3
behavior_df.drop(behavior_df.index[[i for i in range(0, len(behavior_df.index)) if i % ROW_MULTIPLE != 0]], inplace=True)

# For the activity Dataframe, we Change column names to corresponding neuron names 
activity_df.columns = ['neuron' + str(i) for i in range(1, len(activity_df.columns) + 1)]

# Change column names to the behavior column names found in the MossyEPM, MATLAB struct
behavior_df.columns = behavior_column_names

# Fix indexing after downsample
behavior_df.reset_index(drop=True, inplace=True)

# Make the behavior Dataframe indexed by Recording time
# behavior_df.set_index('Recording time', inplace=True)

# Make the behavior Dataframe and the activity Dataframe have the same amount of rows 
if len(behavior_df.index) > len(activity_df.index):
    diff = len(behavior_df.index) - len(activity_df.index)
    behavior_df = behavior_df[:-diff]
elif len(behavior_df.index) < len(activity_df.index):
    diff = len(activity_df.index) - len(behavior_df.index)
    activity_df = activity_df[:-diff]

# Define running frames
VELOCITY_CUTOFF = 4;

# Adds column to the end of the behavior Dataframe and make each cell in that column a 0 
# if the corresponding velocity < VELOCITY_CUTOFF or a 1 if the corresponding velocity >= VELOCITY_CUTOFF
behavior_df['Running_frames'] = [1 if velocity > VELOCITY_CUTOFF else 0 for velocity in behavior_df['Velocity'].tolist()]

# Calculate rates by arm, running, and time period

# Divide session into this many bins
TIME_BIN_FACTOR = 3; 

# activity_df
# behavior_df
AUC_dataframe.columns = ['neuron' + str(i) for i in range(1, len(AUC_dataframe.columns)+1)]
result_dataframe = pd.concat([AUC_dataframe, behavior_df], axis=1)
# result_dataframe

In [4]:
# test_lst = bin_dataframe(result_dataframe, 3)
analysis_utils.activity_by_neurons(result_dataframe, 'neuron6', OpenArms_centerpoint=1)

0.75027020368342978

In [5]:
# experiment = analysis_utils.load_Activities_DataFrame(result_dataframe)
# experiment.plot.bar(figsize=(17, 5), rot=0);

In [6]:
# corr_df = cell_transients_dataframe[(cell_transients_dataframe['neuron1'] != 0) & (cell_transients_dataframe['neuron3'] != 0)]
# corr_df

# Find and plot all neurons
foo = cell_transients_dataframe.columns.tolist()
for i in range(0, len(foo)):
    for j in range(i+1, len(foo)):
        if analysis_utils.neuron_scatter_plot_with_reg(foo[i], foo[j], cell_transients_dataframe) != False:
            analysis_utils.neuron_line_plot(foo[i], foo[j], cell_transients_dataframe)

(neuron1, neuron3): r-value = 0.34605520640188864, r^2-value = 0.11975420587785375


(neuron1, neuron6): r-value = 0.3280461366749471, r^2-value = 0.10761426778735807


In [ ]:
# result_dataframe.plot.scatter(x='Trial_time', y='neuron7');

neuron7_list = list()
for i in range(0, result_dataframe['neuron7'].size):
    if result_dataframe['Open2_centerpoint'][i] != 0:
        neuron7_list.append(result_dataframe['neuron7'][i])
    else:
        neuron7_list.append(np.nan)

# neuron7_dataframe = result_dataframe[['Trial time']].copy()
# neuron7_dataframe['neuron7'] = pd.Series(neuron7_list)
# neuron7_dataframe['neuron7'][neuron7_dataframe['neuron7'] > 0]
# neuron7_dataframe['neuron7'] = pd.DataFrame(columns=[n]neuron7_list)
# sns.regplot(x=result_dataframe['Trial time'], y=neuron7_dataframe['neuron7'], color="g");

In [ ]:
FRAME_RATE = 10
run_vector = behavior_df['Running_frames'][behavior_df['Running_frames'] != 0].index.tolist()
no_run_vector = behavior_df['Running_frames'][behavior_df['Running_frames'] == 0].index.tolist()

run_sums = [activity_df[column][run_vector].sum() for column in activity_df]
no_run_sums = [activity_df[column][no_run_vector].sum() for column in activity_df]   

run_sums1 = [FRAME_RATE * run_sums[i] / len(run_vector) for i in range(0, len(run_sums))]
no_run_sums1 = [FRAME_RATE * no_run_sums[i] / len(no_run_vector) for i in range(0, len(no_run_sums))]

run_noruns_dataframe = pd.DataFrame(columns=['runs', 'norun'])
run_noruns_dataframe['runs'] = pd.Series(run_sums1)
run_noruns_dataframe['norun'] = pd.Series(no_run_sums1)
# run_noruns_dataframe.plot.bar(figsize=(17, 5), rot=0);
# data_1 = [Bar(x=run_noruns_dataframe, y=run_noruns_dataframe)]
# plotly.offline.iplot(data_1)

In [ ]:
# Compute the correlation matrix
corr = pd.concat([cell_transients_dataframe, behavior_df], axis=1).corr()

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=0.9, center=0, square=True, linewidths=.5, cbar_kws={"shrink": .5});